In [1]:
import torch

from random import random, randint

from src.agent import Agent
from src.config import Config
from src.state import State
from src.world import World

# Init 
----

In [3]:
config = Config()
world = World.create_from_config(config=config)
state = State.create_from(
    config=config,
    id="earth",
    x=randint(config.world_min_x, config.world_max_x),
    y=randint(config.world_min_y, config.world_max_y),
)
agent = Agent.create_from(id="earth", state=state)